In [5]:
from Bio import AlignIO
import pandas as pd
import re

# Load alignment
alignment = AlignIO.read("alignf_out.fasta", "fasta")

# Step 1: Identify reference sequence
ref = alignment[0]
ref_seq = ref.seq

# Step 2: Map ungapped positions
positions = []
for i, aa in enumerate(ref_seq):
    if aa != "-":
        pos_num = len([r for r in ref_seq[:i] if r != "-"]) + 1
        positions.append((i, aa, pos_num))

# Step 3: Build mutation -> set of sequence IDs
mut_map = {}


# Mapping from seq IDs to variants (based on second dash-separated element)
seq_to_variant = {}

for record in alignment[1:]:
    seq_id = record.id
    header = record.description
    parts = header.split("-")
    variant = parts[0] + "-" + parts[1] if len(parts) > 1 else "Unknown"
    seq_to_variant[seq_id] = variant


    seq = record.seq
    for i, ref_aa, pos in positions:
        # if pos < 319 or pos > 531:
        #     continue
        var_aa = seq[i]
        if var_aa == ref_aa:
            continue
        if var_aa == "-":
            label = f"{ref_aa}{pos}-"
        elif ref_aa == "-":
            label = f"-{pos}{var_aa}"
        elif var_aa == "X":
            continue
        else:
            label = f"{ref_aa}{pos}{var_aa}"
        mut_map.setdefault(label, set()).add(seq_id)

# Step 4: Build dataframe
seq_ids = [rec.id for rec in alignment[1:]]
mut_labels = sorted(mut_map.keys(), key=lambda x: (int(re.findall(r'\d+', x)[0]), x))

data = {'sequence': seq_ids,
        'variant': [seq_to_variant[s] for s in seq_ids]}

for label in mut_labels:
    data[label] = [1 if seq in mut_map[label] else 0 for seq in seq_ids]

df = pd.DataFrame(data)


# Save to CSV
# Generate safe variant tag for filename
variant_tag = sorted(set(seq_to_variant.values()))[0]

# Save to CSV with variants in the filename
df.to_csv(f"mutation/mutation-{variant_tag}.csv", index=False, header=True)



In [6]:
import pandas as pd
import os
mutation_folder = "mutation"
csv_files = [f for f in os.listdir(mutation_folder) if f.endswith('.csv') and "mutation" in f and "all" not in f]
data = pd.DataFrame()
for csv_file in sorted(csv_files):
    data = pd.concat([data, pd.read_csv(os.path.join(mutation_folder, csv_file))])
data.fillna(0, inplace=True)

for col in data.columns:
    if col == "sequence" or col == "variant":
        continue
    data[col] = data[col].astype(int)
data.to_csv("mutation/mutation-all.csv", index=False, header=True)

In [117]:
spike_locations = """
N487(ND2)
K417(NZ)
Q493(NE2)
Y505(OH)
Y449(OH)
T500(OG1)
N501(N)
G446(O)
Y449(OH)
Y489(OH)
N487(OD1)
G502(N)
Y505(OH)
K417(NZ)
K417(NZ)
"""
ace2_locations = """
Q24(OE1)
D30(OD2)
E35(OE2)
E37(OE2)
D38(OD2)
Y41(OH)
Y41(OH)
Q42(NE2)
Q42(NE2)
Y83(OH)
Y83(OH)
K353(O)
R393(NH2)
D30(OD1)
D30(OD2)
"""

spike_residues = set()
for line in spike_locations.split("\n"):
    loc = ""
    for c in line:
        if c in "0123456789":
            loc += c
        elif c == "(":
            break
    if loc:
        spike_residues.add(int(loc) - 319)

spike_residues


{98, 127, 130, 168, 170, 174, 181, 182, 183, 186}

In [118]:
ace2_residues = set()
for line in ace2_locations.split("\n"):
    loc = ""
    for c in line:
        if c in "0123456789":
            loc += c
        elif c == "(":
            break
    if loc:
        ace2_residues.add(int(loc))
ace2_residues


{24, 30, 35, 37, 38, 41, 42, 83, 353, 393}

In [ ]:

ACE2: 24, 30, 35, 37, 38, 41, 42, 83, 353, 393

Spike: 98, 127, 130, 168, 170, 174, 181, 182, 183, 186